# Декораторы

In [ ]:
def foo(x):
    print(x + 1)


def decorator(foo):
    def _dec(*args, **kwargs):
        print("inside decorator")
        res = foo(*args, **kwargs)
        return res
    
    return _dec

In [ ]:
foo(1)

In [ ]:
foo = decorator(foo)
foo(1)

### В Python есть специальный синтаксис, который позволяет переместить модификацию функции ближе к моменту объявления функции:

In [ ]:
@decorator
def foo(x):
    """
    foo description
    """
    print(x + 1)

In [ ]:
foo(1)

### Есть небольшая проблема:

In [ ]:
foo.__name__, foo.__doc__

#### Еще "пропадает" ```__module__```
#### Что делать?

In [ ]:
def decorator(foo):
    def _dec(*args, **kwargs):
        res = foo(*args, **kwargs)
        return res
    
    _dec.__name__ = foo.__name__
    _dec.__doc__ = foo.__doc__
    _dec.__module__ = foo.__module__
    
    return _dec


@decorator
def foo(x):
    """
    foo description
    """
    print(x + 1)

In [ ]:
foo.__name__, foo.__doc__

#### Но это сложно, можно проще.
#### Воспользуемся декоратором ```wraps``` из пакета ```functools``` <i>(подробнее про этот пакет чуть позже)</i>

In [ ]:
from functools import wraps


def decorator(foo):
    @wraps(foo)
    def _dec(*args, **kwargs):
        res = foo(*args, **kwargs)
        return res
    
    return _dec


@decorator
def foo(x):
    """
    foo description
    """
    print(x + 1)

In [ ]:
foo.__name__, foo.__doc__

### Декораторов может быть несколько:

In [ ]:
def dec1(func):
    @wraps(func)
    def _dec(*args, **kwargs):
        print("inside dec1")
        res = func(*args, **kwargs)
        return res
        
    return _dec


def dec2(func):
    @wraps(func)
    def _dec(*args, **kwargs):
        print("inside dec2")
        res = func(*args, **kwargs)
        return res
        
    return _dec

In [ ]:
@dec2
@dec1
def func(arg1, arg2):
    pass

In [ ]:
func(1, 2)

#### Это же самое, что:

In [ ]:
def func(arg1, arg2):
    pass

In [ ]:
func = dec2(dec1(func))

In [ ]:
func(1, 2)

### Вопрос [?]

In [ ]:
flag = False


def decorator(func):
    @wraps(func)
    def _dec(*args, **kwargs):
        print("inside _dec")
        res = func(*args, **kwargs)
        return res
    
    return func if flag else _dec


@decorator
def func():
    pass

In [ ]:
func()

#### Что будет?

In [ ]:
flag = True
func()

### В декоратор можно передавать аргументы:

In [ ]:
def decorator_with_args(dec_argument):
    def _decorator(func):
        @wraps(func)
        def _dec(*args, **kwargs):
            print(f"inside decorator; {dec_argument}")
            res = func(*args, **kwargs)
            return res
        
        return _dec
    return _decorator

In [ ]:
@decorator_with_args("hop hey lala ley")
def func(x):
    pass

In [ ]:
func(None)

#### Это то же самое, что:

In [ ]:
def decorator_with_args(dec_argument):
    def _decorator(func):
        @wraps(func)
        def _dec(*args, **kwargs):
            print(f"inside decorator; {dec_argument}")
            res = func(*args, **kwargs)
            return res
        
        return _dec
    return _decorator

In [ ]:
def func(x):
    pass

In [ ]:
decorator = decorator_with_args("hop hey lala ley")  # получим тут декоратор
func = decorator(func)  # получим модифицированную функцию

In [ ]:
func(None)

#### А если мы хотим опциональные аргументы?

In [ ]:
def decorator_with_optional_arguments(func=None, *, dec_argument="default"):
    if func is None:
        return lambda func: decorator_with_optional_arguments(func, dec_argument=dec_argument)
    @wraps(func)
    def _dec(*args, **kwargs):
        print(f"inside decorator; {dec_argument}")
        res = func(*args, **kwargs)
        return res
    
    return _dec

In [ ]:
@decorator_with_optional_arguments(dec_argument="Life is beatiful")
def func1():
    pass


func1()

In [ ]:
@decorator_with_optional_arguments
def func2():
    pass


func2()

### Несколько примеров полезных декораторов

#### Посчитать, сколько раз выполнялась функция:

In [ ]:
def profiled(func):
    @wraps(func)
    def inner(*args, **kwargs):
        inner.ncalls += 1
        return func(*args, **kwargs)
    inner.ncalls = 0
    return inner

In [ ]:
@profiled
def f():
    pass


for i in range(1000):
    f()
    
    
print(f.ncalls)

#### Декоратор, для вызова декорируемой функции только 1 раз

In [ ]:
# TODO
def once(func):
    pass

* [contextlib.contextmanager](https://docs.python.org/3.5/library/contextlib.html#contextlib.contextmanager)
* [functools.lru_cache](https://docs.python.org/3.5/library/functools.html#functools.lru_cache)
* почти все в [pycontracts](https://andreacensi.github.io/contracts/)
* @classmethod и @staticmethod
* @property
* @abstractmethod

...

# Области видимости

In [ ]:
print(min)  # build-in

a = 0  # global
b = 1  # global


def g():
    a = -1  # enclosing
    b = -2  # enclosing
    
    def f():
        a = 1  # local
        b = 2  # local

        print("locals: ", locals())

        print("a in globals=", globals()["a"], sep="")
        print("b in globals=", globals()["b"], sep="")
        
    f()
    
    
g()

### А что с присваиванием?

In [ ]:
a = -1  # global


def f():
    try:
        a += 1  # local
    except UnboundLocalError:
        print("Catch UnboundLocalError")


f()

#### Можно ли как-то присвоить? Да!

In [ ]:
a = -1  # global


def f():
    globals()["a"] += 1  # local


f()
print(a)

#### Или так:

In [ ]:
a = -1  # global


def f():
    global a
    a += 1  # local
    print(a)


f()

#### Для ```enclosing```, соответственно, вот так вот:

In [ ]:
def g():
    a = -1  # enclosing

    def f():
        nonlocal a
        a += 1  # local

    f()
    print(a)
    
    
g()

# Дескрипторы

### @property

##### Где ошибка?

In [ ]:
class MoneySaver:
    def __init__(self, exchange_rate):
        self.exchange_rate = exchange_rate
        self.copilka = 0
        
    @property
    def copilka(self):
        return self.copilka * self.exchange_rate
    
    @copilka.setter
    def copilka(self, x):
        self.copilka += x / self.exchange_rate

In [ ]:
saver = MoneySaver(100)
saver.copilka = 10
print(saver.copilka)

##### Тут без ошибок :)

In [ ]:
class SomeBank:
    def __init__(self, start):
        self._rubles = start
        
    @property
    def rubles(self):
        return self._rubles
    
    @rubles.setter
    def rubles(self, value):
        if value > 0:
            self._rubles = value
        else:
            raise Exception('You shall not pass!')

    @rubles.deleter
    def rubles(self, value):
        del self._rubles

In [ ]:
interv = SomeBank(6000)

In [ ]:
interv.rubles

In [ ]:
interv.rubles = 4242

In [ ]:
interv.rubles = -32

### \_\_get__(), \_\_set__() и \_\_delete__(), 

In [ ]:
class Descr:
    def __get__(self, instance, owner):
        print(instance, owner)
        
    def __set__(self, instance, value):
        print(instance, value)
        
    def __delete__(self, instance):
        print(instance)
        
    
class A:
    attr = Descr()

In [ ]:
A().attr

In [ ]:
A.attr

In [ ]:
A().attr = 42

#### Что будет?

In [ ]:
A.attr = 42

In [ ]:
del A().attr

In [ ]:
del A.attr

### Хм

* `instance` -- экз. класса
* `attr` -- атрибут (который дескриптор)
* `descr = cls.__dict__["attr"]` -- сам дескриптор

In [ ]:
instance = A()
descr = A.__dict__["attr"]

####  Тогда:

In [ ]:
A.attr
descr.__get__(None, A)

In [ ]:
instance.attr
descr.__get__(instance, A)

In [ ]:
instance.attr = 42
descr.__set__(instance, 42)

In [ ]:
del instance.attr
descr.__delete__(instance)

### Еще Вопрос [?]

#### Как вы думаете, как работают методы в Python?

In [ ]:
from types import MethodType


class Function(object):
    # ...
    def __get__(self, obj, objtype=None):
        "Simulate func_descr_get() in Objects/funcobject.c"
        if obj is None:
            return self
        return MethodType(self, obj)

In [ ]:
MethodType(object, lambda: 1)

In [ ]:
class Foo(object):
     def bar(self, arg1, arg2):
            print(arg1, arg2)

foo = Foo()
# this:
foo.bar(1,2)
# does about the same thing as this:
Foo.__dict__['bar'].__get__(foo, type(foo))(1,2)

`types.MethodType`

* http://stupidpythonideas.blogspot.com/2013/06/how-methods-work.html
* http://igorsobreira.com/2011/02/06/adding-methods-dynamically-in-python.html